<a href="https://colab.research.google.com/github/jyshin0926/Text-Calssification/blob/master/google_research_bigbird_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install git+https://github.com/google-research/bigbird.git -q

     |████████████████████████████████| 1.2MB 12.7MB/s 
     |████████████████████████████████| 3.4MB 27.6MB/s 
     |████████████████████████████████| 1.5MB 53.6MB/s 
     |████████████████████████████████| 3.8MB 53.4MB/s 
     |████████████████████████████████| 194kB 56.3MB/s 
     |████████████████████████████████| 368kB 49.4MB/s 
     |████████████████████████████████| 81kB 11.3MB/s 
     |████████████████████████████████| 655kB 50.8MB/s 
     |████████████████████████████████| 368kB 42.2MB/s 
     |████████████████████████████████| 983kB 51.8MB/s 
     |████████████████████████████████| 5.6MB 54.1MB/s 
     |████████████████████████████████| 706kB 53.5MB/s 
     |████████████████████████████████| 358kB 52.0MB/s 
     |████████████████████████████████| 163kB 58.6MB/s 
     |████████████████████████████████| 256kB 56.5MB/s 
ERROR: kfac 0.2.3 has requirement tensorflow-probability==0.8, but you'll have tensorflow-probability 0.7.0 which is incompatible.


In [ ]:
from bigbird.core import flags
from bigbird.core import modeling
from bigbird.core import utils
from bigbird.classifier import run_classifier
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
from tqdm import tqdm
import sys

FLAGS = flags.FLAGS
if not hasattr(FLAGS, "f"): flags.DEFINE_string("f", "", "")
FLAGS(sys.argv)

tf.enable_v2_behavior() 

In [ ]:
FLAGS.data_dir = "tfds://imdb_reviews/plain_text"
FLAGS.attention_type = "block_sparse"
FLAGS.max_encoder_length = 3072  # 4096 on 16GB GPUs like V100, on free colab only lower memory GPU like T4 is available
FLAGS.learning_rate = 1e-5
FLAGS.num_train_steps = 10000
FLAGS.attention_probs_dropout_prob = 0.0
FLAGS.hidden_dropout_prob = 0.0
FLAGS.vocab_model_file = "gpt2"

In [ ]:
bert_config = flags.as_dictionary()

In [ ]:
model = modeling.BertModel(bert_config)
headl = run_classifier.ClassifierLossLayer(
        bert_config["num_labels"], bert_config["hidden_dropout_prob"],
        utils.create_initializer(bert_config["initializer_range"]),
        name=bert_config["scope"]+"/classifier")

In [ ]:
@tf.function(experimental_compile=True)
def fwd_bwd(features, labels):
  with tf.GradientTape() as g:
    _, pooled_output = model(features, training=True)
    loss, log_probs = headl(pooled_output, labels, True)
  grads = g.gradient(loss, model.trainable_weights+headl.trainable_weights)
  return loss, log_probs, grads

In [ ]:
train_input_fn = run_classifier.input_fn_builder(
        data_dir=FLAGS.data_dir,
        vocab_model_file=FLAGS.vocab_model_file,
        max_encoder_length=FLAGS.max_encoder_length,
        substitute_newline=FLAGS.substitute_newline,
        is_training=True)
dataset = train_input_fn({'batch_size': 2})

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


/usr/local/lib/python3.7/dist-packages/bigbird/classifier/run_classifier.py:165: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  deterministic=is_training)


In [ ]:
# inspect at a few examples
for ex in dataset.take(3):
  print(ex)

(<tf.Tensor: shape=(2, 3072), dtype=int32, numpy=
array([[   65,   733,   474, ...,     0,     0,     0],
       [   65,   415, 26500, ...,     0,     0,     0]], dtype=int32)>, <tf.Tensor: shape=(2,), dtype=int32, numpy=array([0, 1], dtype=int32)>)
(<tf.Tensor: shape=(2, 3072), dtype=int32, numpy=
array([[   65,   484, 20677, ...,     0,     0,     0],
       [   65,   871,  3908, ...,     0,     0,     0]], dtype=int32)>, <tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 1], dtype=int32)>)
(<tf.Tensor: shape=(2, 3072), dtype=int32, numpy=
array([[  65,  415, 6506, ...,    0,    0,    0],
       [  65,  418, 1150, ...,    0,    0,    0]], dtype=int32)>, <tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 0], dtype=int32)>)


In [ ]:
## pretrained model 사용하기
ckpt_path = 'gs://bigbird-transformer/pretrain/bigbr_base/model.ckpt-0'
ckpt_reader = tf.compat.v1.train.NewCheckpointReader(ckpt_path)
model.set_weights([ckpt_reader.get_tensor(v.name[:-2]) for v in tqdm(model.trainable_weights, position=0)])

100%|██████████| 199/199 [00:42<00:00,  4.65it/s]


In [ ]:
# 직접 학습시키기
opt = tf.keras.optimizers.Adam(FLAGS.learning_rate)
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

for i, ex in enumerate(tqdm(dataset.take(FLAGS.num_train_steps), position=0)):
  loss, log_probs, grads = fwd_bwd(ex[0], ex[1])
  opt.apply_gradients(zip(grads, model.trainable_weights+headl.trainable_weights))
  train_loss(loss)
  train_accuracy(tf.one_hot(ex[1], 2), log_probs)
  if i% 1000 == 0:
    print('Loss = {}  Accuracy = {}'.format(train_loss.result().numpy(), train_accuracy.result().numpy()))

  0%|          | 0/10000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py:1402: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '
  0%|          | 1/10000 [01:06<185:40:44, 66.85s/it]

Loss = 0.6948387622833252  Accuracy = 0.5


 10%|█         | 1001/10000 [20:48<2:57:25,  1.18s/it]

Loss = 0.7104178667068481  Accuracy = 0.5144855380058289


 20%|██        | 2001/10000 [40:32<2:38:45,  1.19s/it]

...

 80%|████████  | 8001/10000 [2:38:39<39:08,  1.18s/it]

Loss = 0.6994261741638184  Accuracy = 0.5046244263648987


 90%|█████████ | 9001/10000 [2:58:18<19:37,  1.18s/it]

Loss = 0.6992313265800476  Accuracy = 0.5016109347343445


100%|██████████| 10000/10000 [3:17:57<00:00,  1.19s/it]


## Evaluation

In [ ]:
@tf.function(experimental_compile=True)
def fwd_only(features, labels):
  _, pooled_output = model(features, training=False)
  loss, log_probs = headl(pooled_output, labels, False)
  return loss, log_probs

In [ ]:
eval_input_fn = run_classifier.input_fn_builder(
        data_dir=FLAGS.data_dir,
        vocab_model_file=FLAGS.vocab_model_file,
        max_encoder_length=FLAGS.max_encoder_length,
        substitute_newline=FLAGS.substitute_newline,
        is_training=False)
eval_dataset = eval_input_fn({'batch_size': 2})

/usr/local/lib/python3.7/dist-packages/bigbird/classifier/run_classifier.py:165: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  deterministic=is_training)


In [ ]:
## 직접 학습시켰을 때
eval_loss = tf.keras.metrics.Mean(name='eval_loss')
eval_accuracy = tf.keras.metrics.CategoricalAccuracy(name='eval_accuracy')

for ex in tqdm(eval_dataset, position=0):
  loss, log_probs = fwd_only(ex[0], ex[1])
  eval_loss(loss)
  eval_accuracy(tf.one_hot(ex[1], 2), log_probs)
print(f'Loss: {eval_loss.result().numpy()} Accuracy: {eval_accuracy.result().numpy()}')

  0%|          | 0/12500 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py:1402: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '
100%|██████████| 12500/12500 [1:29:08<00:00,  2.34it/s]

Loss: 0.6934798359870911 Accuracy: 0.5


In [ ]:
## Pretrained model 사용했을 때
eval_loss = tf.keras.metrics.Mean(name='eval_loss')
eval_accuracy = tf.keras.metrics.CategoricalAccuracy(name='eval_accuracy')

for ex in tqdm(eval_dataset, position=0):
  loss, log_probs = fwd_only(ex[0], ex[1])
  eval_loss(loss)
  eval_accuracy(tf.one_hot(ex[1], 2), log_probs)
print(f'Loss: {eval_loss.result().numpy()} Accuracy: {eval_accuracy.result().numpy()}')

  0%|          | 0/12500 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py:1402: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '
100%|██████████| 12500/12500 [1:27:55<00:00,  2.37it/s]

Loss: 0.7981386780738831 Accuracy: 0.5
